In [1]:
from glmsingle.glmsingle import GLM_single
import argparse
import os
import os.path as op
from nilearn import image
from numrisk.utils.data import Subject
from nilearn.glm.first_level import make_first_level_design_matrix
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

bids_folder = '/Volumes/mrenkeED/data/ds-dnumr'

/Users/mrenke/mambaforge/envs/numrefields/lib/python3.10/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [3]:
subject = '01'
session = 1
derivatives = op.join(bids_folder, 'derivatives')

runs = range(1, 7)
runs = range(4,7) # test only half data

sub = Subject(subject, bids_folder=bids_folder)

ims = sub.get_preprocessed_bold(session=session, runs=runs)

base_dir = 'glm_stim1.denoise'

In [4]:
data = [image.load_img(im).get_fdata() for im in ims]

base_dir = op.join(derivatives, base_dir, f'sub-{subject}',
                    f'ses-{session}', 'func')



In [38]:
np.shape(image.load_img(ims[0]).get_fdata())[3]

188

In [5]:
onsets = sub.get_fmri_events(session=session, runs = runs)
print(np.shape(onsets))
onsets.head()

tr = 2.3
n = np.shape(image.load_img(ims[0]).get_fdata())[3]  # before: 135  np.shape(data)[2]
frametimes = np.linspace(tr/2., (n - .5)*tr, n)
onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3

(180, 5)


In [14]:

dm = [make_first_level_design_matrix(frametimes, onsets.loc[run], hrf_model='fir', oversampling=100.,
                                        drift_order=0,
                                        drift_model=None).drop('constant', axis=1) for run in runs]


In [17]:
dm

[        n2_10_delay_0  n2_11_delay_0  n2_14_delay_0  n2_16_delay_0  \
 1.15              0.0            0.0            0.0            0.0   
 3.45              0.0            0.0            0.0            0.0   
 5.75              0.0            0.0            0.0            0.0   
 8.05              0.0            0.0            0.0            0.0   
 10.35             0.0            0.0            0.0            0.0   
 ...               ...            ...            ...            ...   
 422.05            0.0            0.0            0.0            0.0   
 424.35            0.0            0.0            0.0            0.0   
 426.65            0.0            0.0            0.0            0.0   
 428.95            0.0            0.0            0.0            0.0   
 431.25            0.0            0.0            0.0            0.0   
 
         n2_19_delay_0  n2_2_delay_0  n2_23_delay_0  n2_28_delay_0  \
 1.15              0.0           0.0            0.0            0.0   
 3.45 

In [9]:
dm = pd.concat(dm, keys=runs, names=['run']).fillna(0)
dm.columns = [c.replace('_delay_0', '') for c in dm.columns]
dm /= dm.max()
#print(dm)
dm[dm < 1.0] = 0.0
print(dm.shape)

n2_10  n2_11  n2_14  n2_16  n2_19  n2_2  n2_23  n2_28  n2_3  n2_33  \
run                                                                             
1   1.15     0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0   
    3.45     0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0   
    5.75     0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0   
    8.05     0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0   
    10.35    0.0    0.0    0.0    0.0    0.0   0.0    0.0    0.0   0.0    0.0   

           ...  trial_171_n1  trial_172_n1  trial_173_n1  trial_174_n1  \
run        ...                                                           
1   1.15   ...           0.0           0.0           0.0           0.0   
    3.45   ...           0.0           0.0           0.0           0.0   
    5.75   ...           0.0           0.0           0.0           0.0   
    8.05   ...           0.0           0.0           0.0           0.0   
    10.35  ...           0.0           0.0           0.0           0.0   

           trial_175_n1  trial_176_n1  trial_177_n1  trial_178_n1  \
run                                                                 
1   1.15            0.0           0.0           0.0           0.0   
    3.45            0.0           0.0           0.0           0.0   
    5.75            0.0           0.0           0.0           0.0   
    8.05            0.0           0.0           0.0           0.0   
    10.35           0.0           0.0           0.0           0.0   

           trial_179_n1  trial_180_n1  
run                                    
1   1.15            0.0           0.0  
    3.45            0.0           0.0  
    5.75            0.0           0.0  
    8.05            0.0           0.0  
    10.35           0.0           0.0  

[5 rows x 110 columns]

In [8]:
X = [dm.loc[run].values for run in runs]

opt = dict()
opt['wantlibrary'] = 1
opt['wantglmdenoise'] = 1
opt['wantfracridge'] = 1
opt['wantfileoutputs'] = [0, 0, 0, 1]


KeyError: 4

In [ ]:
glmsingle_obj = GLM_single(opt)

results_glmsingle = glmsingle_obj.fit(
    X,
    data,
    0.6,
    2.3,
    outputdir=base_dir)

betas = results_glmsingle['typed']['betasmd']
betas = image.new_img_like(ims[0], betas)
betas = image.index_img(betas, slice(None, None, 2))

In [10]:
np.shape(X)

NameError: name 'X' is not defined

In [12]:
print(dm.shape)

(564, 110)


In [11]:
np.shape(data)

(3, 59, 71, 49, 188)

In [18]:
ims

['/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-1_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-2_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-3_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-4_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-5_space-T1w_desc-preproc_bold.nii.gz',
 '/Volumes/mrenkeED/data/ds-dnumr/derivatives/fmriprep/sub-01/ses-1/func/sub-01_ses-1_task-magjudge_run-6_space-T1w_desc-preproc_bold.nii.gz']

In [21]:
np.shape(onsets)

(366, 5)

In [22]:
366/6

61.0

In [18]:
split_data = None
base_dir = f'glm_stim1{split_data}.denoise'
base_dir

'glm_stim1None.denoise'